In [1]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import sproc,sum

import snowflake.snowpark
from snowflake.snowpark import functions as F

def initiateSession(): 
    connection_parameters = {
            "account": "ls88978.eu-west-1",
            "user": "snowpark_user",
            "password": "UdemySnowflake12!@",
            "role": "Accountadmin", 
            "warehouse": "compute_wh",
            "database":"SNOWPARK_UDEMY",
            "schema":"TEST_SCHEMA"
    }
    session = Session.builder.configs(connection_parameters).create()
    return session

session = initiateSession()

session.add_packages('snowflake-snowpark-python', 'pandas')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [2]:

query = "create or replace stage snowpark_sproc_stage" +\
        " directory = (enable = true)" +\
        " copy_options = (on_error='skip_file')"
session.sql(query).collect()

[Row(status='Stage area SNOWPARK_SPROC_STAGE successfully created.')]

<H2>Create Stored Procedure Function</H2>

In [3]:
def df_transform_sproc(session: snowflake.snowpark.Session) -> str:
    
    df_orders = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.ORDERS") \
                .select(['O_ORDERKEY','O_CUSTKEY','O_TOTALPRICE']) \
                .filter("O_TOTALPRICE>=100000")

    df_lineitem = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.LINEITEM") \
                .select(['L_SUPPKEY','L_ORDERKEY','L_QUANTITY','L_PARTKEY'])

    df_order_lineitem = df_orders.join(df_lineitem,df_lineitem.L_ORDERKEY == df_orders.O_ORDERKEY) \
            .select(['L_ORDERKEY','O_CUSTKEY','O_TOTALPRICE','L_QUANTITY','L_SUPPKEY','L_PARTKEY'])
    
    try : 
        df_order_lineitem.write.mode("overwrite").save_as_table("order_lineitems")
        return 'success'
    except Exception as e : 
        return e

<H2>Create a Table in SF To Store the result of above Transformation</H2>

In [4]:
query = """create or replace table SNOWPARK_UDEMY.test_schema.order_lineitems 
        (
            L_ORDERKEY integer,O_CUSTKEY integer,O_TOTALPRICE float ,L_QUANTITY FLOAT,
            L_SUPPKEY integer,L_PARTKEY integer
        )
        """

session.sql(query).collect()

[Row(status='Table ORDER_LINEITEMS successfully created.')]

<h2>Deploy the Stored Proc</h2>

In [5]:
execute_dftransforms_sproc = sproc(df_transform_sproc,name='df_transform_sproc',stage_location='@snowpark_sproc_stage', \
                               is_permanent=True,replace=True)


<H2>Invoke Stored Proc</H2>

In [6]:
execute_dftransforms_sproc()

'success'

<h2>Create Stored Proc using decorators</h2>

In [7]:
@sproc(name="customer_order_qty_sproc", replace=True, is_permanent=True, stage_location="@snowpark_sproc_stage")

def customer_order_qty_func(session: snowflake.snowpark.Session,cust_key:int) -> dict:
    
    df_orders = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.ORDERS") \
                .select(['O_ORDERKEY','O_CUSTKEY','O_TOTALPRICE']) \
                .filter("O_CUSTKEY="+str(cust_key))
    
    df_lineitem = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.LINEITEM") \
                .select(['L_SUPPKEY','L_ORDERKEY','L_QUANTITY'])
    
    df_lineitem = df_lineitem.groupBy('L_ORDERKEY') \
            .sum(df_lineitem.L_QUANTITY).withColumnRenamed("SUM(L_QUANTITY)","sum_qty")
    

    df_order_details = df_orders.join(df_lineitem,df_lineitem.L_ORDERKEY == df_orders.O_ORDERKEY) \
            .select(['L_ORDERKEY','O_TOTALPRICE','SUM_QTY'])
    
    df_order_details = df_order_details.to_pandas()
    
    return df_order_details.to_dict()

<h2>Invoke Stored Proc in the current Session </h2>

In [8]:
session.call("customer_order_qty_sproc",171983)

'{\n  "L_ORDERKEY": {\n    "0": 6397767,\n    "1": 45389760,\n    "10": 33993218,\n    "11": 54065831,\n    "12": 13363936,\n    "13": 28651620,\n    "14": 17163012,\n    "15": 25825507,\n    "16": 43183713,\n    "17": 17904067,\n    "18": 45069253,\n    "19": 59000928,\n    "2": 13235332,\n    "20": 32082881,\n    "3": 57159842,\n    "4": 46284291,\n    "5": 14379168,\n    "6": 26488259,\n    "7": 22592805,\n    "8": 23173127,\n    "9": 44436551\n  },\n  "O_TOTALPRICE": {\n    "0": 37354.42,\n    "1": 127356.05,\n    "10": 215438.6,\n    "11": 141486.98,\n    "12": 147898.76,\n    "13": 117162.78,\n    "14": 60943.18,\n    "15": 264466.05,\n    "16": 368012.9,\n    "17": 164632.53,\n    "18": 96684.88,\n    "19": 109329.26,\n    "2": 155972.04,\n    "20": 342323.48,\n    "3": 245405.02,\n    "4": 145017.45,\n    "5": 244378.28,\n    "6": 208696.56,\n    "7": 135090.93,\n    "8": 18194.1,\n    "9": 155849.78\n  },\n  "SUM_QTY": {\n    "0": 22,\n    "1": 92,\n    "10": 162,\n    "11": 1